# Two Layer Router - Reduced Measurements

In [1]:
import sys
sys.path.append('..') # Search for custom module in the top level. 

# Import my custom modules.
from allens_quantum_package.functions import * 
from allens_quantum_package.operators import *

from qiskit import *
from qiskit.quantum_info import state_fidelity
from qiskit_ibm_runtime import QiskitRuntimeService

import scipy
import numpy
from numpy import set_printoptions, radians, ndarray, radians, array, sqrt

import itertools

In [2]:
# Set the floating point diplay precision to 3 decimal places, sufficient for our purposes.
set_printoptions(precision=3)

# Initialise the Qiskit runtime service. 
service = QiskitRuntimeService()

In [3]:
states = [
    (radians(-76), radians(-88)),
    (radians(93), radians(-135)),
    (radians(104), radians(-96))
]

In [4]:
def build_circuit(theta, phi) -> QuantumCircuit:
    circ = QuantumCircuit(7, 4)
    
    # Route on first layer
    circ.h(3)
    circ.u(theta, phi, 0, 2)
    circ.cswap(3, 2, 4)
    
    # Route on second layer, top
    circ.h(0)
    circ.cswap(0, 1, 2)
    
    # Route on second layer, bottom
    circ.h(6)
    circ.cswap(6, 4, 5)

    return circ

In [5]:
build_circuit(*states[0]).draw())

SyntaxError: unmatched ')' (2693411349.py, line 1)

In [6]:
def add_basis_meas(circuit, qubit, basis='z') -> QuantumCircuit:
    circ = circuit.copy()
    circ.barrier()

    if basis == 'x':
        circ.h(qubit)
    elif basis == 'y':
        circ.sdg(qubit)
        circ.h(qubit)

    circ.barrier()

    circ.measure([0, 3, 6] + [qubit], [0, 1, 2, 3])

    return circ


# Create named tuple object, for easy indexing, e.g. ".x", instead of "[0]"
TomographySet = namedtuple('TomographySet', ['x', 'y', 'z'])


def get_tomography_circuits(circuit, qubit) -> list[QuantumCircuit]:
    return [
        add_basis_meas(circuit, qubit, 'x'),
        add_basis_meas(circuit, qubit, 'y'),
        add_basis_meas(circuit, qubit, 'z')
    ]

In [7]:
get_tomography_circuits(build_circuit(*states[0]), 2)[2].draw()

┌───┐                 ░  ░ ┌─┐         
q_0: ─────────┤ H ├──────────────■──░──░─┤M├─────────
              └───┘              │  ░  ░ └╥┘         
q_1: ────────────────────────────X──░──░──╫──────────
     ┌──────────────────────┐    │  ░  ░  ║       ┌─┐
q_2: ┤ U(-1.3265,-1.5359,0) ├─X──X──░──░──╫───────┤M├
     └────────┬───┬─────────┘ │     ░  ░  ║ ┌─┐   └╥┘
q_3: ─────────┤ H ├───────────■─────░──░──╫─┤M├────╫─
              └───┘           │     ░  ░  ║ └╥┘    ║ 
q_4: ─────────────────────────X──X──░──░──╫──╫─────╫─
                                 │  ░  ░  ║  ║     ║ 
q_5: ────────────────────────────X──░──░──╫──╫─────╫─
              ┌───┐              │  ░  ░  ║  ║ ┌─┐ ║ 
q_6: ─────────┤ H ├──────────────■──░──░──╫──╫─┤M├─╫─
              └───┘                 ░  ░  ║  ║ └╥┘ ║ 
c: 4/═════════════════════════════════════╩══╩══╩══╩═
                                          0  1  2  3

In [8]:
def generate_tomography_sets(theta, phi) -> list[list[QuantumCircuit]]:
    return [ 
        *get_tomography_circuits(
            build_circuit(theta, phi), 1
        ),
        *get_tomography_circuits(
            build_circuit(theta, phi), 2
        ),
        *get_tomography_circuits(
            build_circuit(theta, phi), 4
        ),
        *get_tomography_circuits(
            build_circuit(theta, phi), 5
        )
    ]

In [9]:
ibm_brisbane = service.get_backend('ibm_brisbane')

In [10]:
circuits_to_send = list(itertools.chain.from_iterable([
        circuit for circuit in [
            itertools.chain.from_iterable(generate_tomography_sets(theta, phi) for theta, phi in states)
        ]
    ]
))

circuits_to_send = [transpile(circuit, ibm_brisbane) for circuit in circuits_to_send]

In [11]:
def get_all_physical_qubits(circuit_list: list[QuantumCircuit]) -> set[int]:
    all_indices = set()
    for circ in circuit_list:    
        measurement_indices = [instr[1][0]._index for instr in circ.data if instr[0].name == 'measure']
        all_indices = all_indices.union(measurement_indices)
    return all_indices


def get_mitigation_circuits(circuit_list: list[QuantumCircuit]) -> list[QuantumCircuit]:

    physical_qubits = get_all_physical_qubits(circuit_list)

    num_qubits = len(physical_qubits)
    all_0 = QuantumCircuit(127, num_qubits)
    all_0.measure(physical_qubits, range(num_qubits))

    all_1 = QuantumCircuit(127, num_qubits)
    all_1.x(physical_qubits)
    all_1.measure(physical_qubits, range(num_qubits))

    return [all_0, all_1]

In [12]:
circuits_to_send = get_mitigation_circuits(circuits_to_send) + circuits_to_send

In [13]:
hardware_job = ibm_brisbane.run(circuits=circuits_to_send)

In [14]:
print(f"Hardware job ID: {hardware_job.job_id()}")

Hardware job ID: cw6fn64jyrs0008gj7jg


---

In [15]:
results = hardware_job.result().get_counts()

In [16]:
def get_combined_xyz_counts_for_circuit(counts_list: list) -> tuple[dict]:

    output_qubit_1 = []
    for counts in counts_list[0:3]:
        counts_dict = {}

        counts_dict['0'] = sum(count for bit_string, count in counts.items() if bit_string[2:4] == '01' and bit_string[0] == '0')
        counts_dict['1'] = sum(count for bit_string, count in counts.items() if bit_string[2:4] == '01' and bit_string[0] == '1')

        output_qubit_1.append(counts_dict)

    output_qubit_2 = []
    for counts in counts_list[3:6]:
        counts_dict = {}
        
        counts_dict['0'] = sum(count for bit_string, count in counts.items() if bit_string[2:4] == '00' and bit_string[0] == '0')
        counts_dict['1'] = sum(count for bit_string, count in counts.items() if bit_string[2:4] == '00' and bit_string[0] == '1')    

        output_qubit_2.append(counts_dict)

    output_qubit_4 = []
    for counts in counts_list[6:9]:
        counts_dict = {}
        
        counts_dict['0'] = sum(count for bit_string, count in counts.items() if bit_string[1:3] == '01' and bit_string[0] == '0')
        counts_dict['1'] = sum(count for bit_string, count in counts.items() if bit_string[1:3] == '01' and bit_string[0] == '1')    
    
        output_qubit_4.append(counts_dict)
    
    output_qubit_5 = []
    for counts in counts_list[9:12]:
        counts_dict = {}
        
        counts_dict['0'] = sum(count for bit_string, count in counts.items() if bit_string[1:3] == '11' and bit_string[0] == '0')
        counts_dict['1'] = sum(count for bit_string, count in counts.items() if bit_string[1:3] == '11' and bit_string[0] == '1')    
    
        output_qubit_5.append(counts_dict)

    output = (add_dicts(dict_1, dict_2, dict_4, dict_5) for dict_1, dict_2, dict_4, dict_5 in zip(output_qubit_1, output_qubit_2, output_qubit_4, output_qubit_5))   
    
    return tuple(output)


def print_unmitigated_fidelities_combined(counts, theta, phi):

    qubit = density_op_from_counts_dict(*get_combined_xyz_counts_for_circuit(counts))

    print(get_combined_xyz_counts_for_circuit(counts))
    print(qubit)

    psi = gen_qubit(theta, phi)

    fidelity_q = state_fidelity(qubit, psi)

    print(f'State fidelity from combined counts: {fidelity_q}\n')

In [17]:
print_unmitigated_fidelities_combined(results[2:14], *states[0])
print_unmitigated_fidelities_combined(results[14:26], *states[1])
print_unmitigated_fidelities_combined(results[26:38], *states[2])

({'0': 2334, '1': 1576}, {'0': 2464, '1': 1561}, {'0': 2322, '1': 1655})
[[0.584+0.j    0.097-0.112j]
 [0.097+0.112j 0.416+0.j   ]]
State fidelity from combined counts: 0.6257800961516129

({'0': 1895, '1': 2140}, {'0': 1537, '1': 2462}, {'0': 2575, '1': 1704})
[[ 0.602+0.j    -0.03 +0.116j]
 [-0.03 -0.116j  0.398+0.j   ]]
State fidelity from combined counts: 0.5977789259985883

({'0': 2009, '1': 1864}, {'0': 1491, '1': 2485}, {'0': 1893, '1': 2006})
[[0.486+0.j    0.019+0.125j]
 [0.019-0.125j 0.514+0.j   ]]
State fidelity from combined counts: 0.6222296266303293



In [18]:
# Gets the map from a physical qubit to a classical bit for the mitigation calibration matrices
def get_qubit_to_clbit_mappings_for_mitigation(circuit: QuantumCircuit) -> dict[int, int]:
    return dict(
        ((instr[1][0]._index, instr[2][0]._index) for instr in circuit.data if instr[0].name == 'measure')
    )

# Gets the map from physical qubit to mitigation matrix
def get_assignment_matrices(mappings: dict, counts_0: dict, counts_1: dict) -> dict[int, ndarray]:

    output = {}

    for physical_qubit, classical_bit in mappings.items():
        
        # Determine zero state for assignment matrix
        result_0 = sum([count for bit_string, count in counts_0.items() if bit_string[classical_bit] == '0'])
        result_1 = sum([count for bit_string, count in counts_0.items() if bit_string[classical_bit] == '1'])
        
        # Calculate zero ket
        zero_ket = array([[result_0], 
                          [result_1]]) / (result_0 + result_1)
        
        # Determine one state for assignment matrix
        result_0 = sum([count for bit_string, count in counts_1.items() if bit_string[classical_bit] == '0'])
        result_1 = sum([count for bit_string, count in counts_1.items() if bit_string[classical_bit] == '1'])
        
        # Calculate zero ket
        one_ket = array([[result_0], 
                         [result_1]]) / (result_0 + result_1)
        
        assignment_matrix = numpy.concatenate([zero_ket, one_ket], axis=1)

        output[physical_qubit] = assignment_matrix
    
    return output

# Get the list of physical qubits in the same order as the classical bits 
def get_qubits_in_clbit_order(circuit: QuantumCircuit) -> list[int]:
    
    cl_bit_to_qbit_map = dict((instr[2][0]._index, instr[1][0]._index) for instr in circuit.data if instr[0].name == 'measure')

    return [*cl_bit_to_qbit_map.values()]
    

# Get the tensor of all mitigation matrices in the correct order for each circuit
def get_mitigation_matrix(circuit: QuantumCircuit, assignment_mappings: dict) -> ndarray:
    ordered_physical_qubits = get_qubits_in_clbit_order(circuit)

    ordered_physical_qubits.reverse()

    assignment_matrices = [assignment_mappings[qubit] for qubit in ordered_physical_qubits]

    return tens(*(scipy.linalg.inv(mat) for mat in assignment_matrices))


def get_bit_strings(count: int) -> list[str]:
    return [''.join(bits) for bits in itertools.product(['0', '1'], repeat=count)]


def get_corrected_counts(circuit: QuantumCircuit, counts_for_mitigation: dict, assignment_mappings: dict, num_qubits: int) -> dict[str, int]:
    bit_strings = get_bit_strings(num_qubits)

    counts_list = []
    for bit_string in bit_strings:
        counts_list.append(counts_for_mitigation[bit_string] if bit_string in counts_for_mitigation else 0)
    
    # Get vector of counts
    counts_vector = numpy.concatenate(
        array([[count for count in counts_list]]),
        axis=0
    )

    mitigation_matrix = get_mitigation_matrix(circuit, assignment_mappings)
    
    # Multiply by mitigation matirx
    corrected_vector = mitigation_matrix @ counts_vector
    corrected_vector = corrected_vector.astype(int)
    
    output = {}
    for idx, bit_string in zip(range(2**num_qubits), bit_strings):
        output[bit_string] = int(corrected_vector[idx])
    
    return output

In [19]:
# Get list of qubit to clbit mappings for the mitigation circuits.
mitigation_mappings = get_qubit_to_clbit_mappings_for_mitigation(circuits_to_send[0])

# Calculate assignment matrices for each qubit. 
assignment_matrices = get_assignment_matrices(mitigation_mappings, results[0], results[1])

In [20]:
def print_mitigated_fidelities_combined(circuits, counts, assignment_matrices, num_qubits, theta, phi):

    mitigated_counts = []

    for circuit, count in zip(circuits, counts):
        mitigated_counts.append(get_corrected_counts(circuit, count, assignment_matrices, num_qubits))

    qubit = density_op_from_counts_dict(*get_combined_xyz_counts_for_circuit(mitigated_counts))
    
    print(get_combined_xyz_counts_for_circuit(mitigated_counts))
    print(qubit)

    psi = gen_qubit(theta, phi)

    fidelity_q = state_fidelity(qubit, psi)

    print(f'Mitigated state fidelity from combined counts: {fidelity_q}\n')

In [21]:
print_mitigated_fidelities_combined(circuits_to_send[2:14], results[2:14], assignment_matrices, 4, *states[0])
print_mitigated_fidelities_combined(circuits_to_send[14:26], results[14:26], assignment_matrices, 4, *states[1])
print_mitigated_fidelities_combined(circuits_to_send[26:38], results[26:38], assignment_matrices, 4, *states[2])

({'0': 2388, '1': 1573}, {'0': 2461, '1': 1531}, {'0': 2297, '1': 1670})
[[0.579+0.j    0.103-0.116j]
 [0.103+0.116j 0.421+0.j   ]]
Mitigated state fidelity from combined counts: 0.6285886859616745

({'0': 1890, '1': 2141}, {'0': 1507, '1': 2492}, {'0': 2521, '1': 1711})
[[ 0.596+0.j    -0.031+0.123j]
 [-0.031-0.123j  0.404+0.j   ]]
Mitigated state fidelity from combined counts: 0.6039411156703309

({'0': 1962, '1': 1863}, {'0': 1442, '1': 2499}, {'0': 1805, '1': 2038})
[[0.47 +0.j    0.013+0.134j]
 [0.013-0.134j 0.53 +0.j   ]]
Mitigated state fidelity from combined counts: 0.6354280662985626

